In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
from sentence_transformers import SentenceTransformer, util
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
import pandas as pd
data = pd.read_excel('C:/Users/HP/Desktop/lowest-common-subsumer/DataPoints.xlsx', index_col=None)
data['word2vec Score']= np.nan
data.head()

,Original,Answers,Points,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,word2vec Score
0,"Mark is at the park. His dog Max, is with him....","Mark is playing with his dog, Max, at the park...",85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Mark is in the park. He is playing with his do...,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Mark and his dog are playing under the sun at ...,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sir Isaac Newton was seated in his garden on a...,It was summer when Newton saw an apple fall fr...,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Sir Isaac Newton was in his garden when an apl...,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import nltk
nltk.download('stopwords')
    # removing special characters and stop words from the text
stop_words_l=stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### word2vec embeddings

In [ ]:
from keras.preprocessing.text import Tokenizer
import gensim
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# loading pre-trained embeddings, each word is represented as a 300 dimensional vector
import gensim

W2V_PATH="C:/Users/HP/Desktop/lowest-common-subsumer/GoogleNews-vectors-negative300.bin"
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH, binary=True)

In [ ]:
results2 = []
m= 0
n=0
while m<300:
    for m in range(m,300):
        original=data['Original'][m]
        #print("original:",original)
        for n in range(1):
            answer1=data['Answers'][m]
            answer2=data['Answers'][m+n+1]
            answer3=data['Answers'][m+n+2]
        break
    documents = [original,answer1,answer2,answer3]
    pd.set_option('display.max_colwidth', 0)
    pd.set_option('display.max_columns', 0)
    documents_df=pd.DataFrame(documents,columns=['documents'])
    documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
    tfidfvectoriser=TfidfVectorizer(max_features=64)
    tfidfvectoriser.fit(documents_df.documents_cleaned)
    tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)
    tfidf_vectors=tfidf_vectors.toarray()
    # tokenize and pad every document to make them of the same size
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(documents_df.documents_cleaned)
    tokenized_documents=tokenizer.texts_to_sequences(documents_df.documents_cleaned)
    tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
    vocab_size=len(tokenizer.word_index)+1
    #print (tokenized_paded_documents[1])

    # creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 
    embedding_matrix=np.zeros((vocab_size,300))
    for word,i in tokenizer.word_index.items():
        if word in model_w2v:
            embedding_matrix[i]=model_w2v[word]
        #print(embedding_matrix[0])

        # creating document-word embeddings
    document_word_embeddings=np.zeros((len(tokenized_paded_documents),64,300))

    for i in range(len(tokenized_paded_documents)):
        for j in range(len(tokenized_paded_documents[0])):
            document_word_embeddings[i][j]=embedding_matrix[tokenized_paded_documents[i][j]] 
            #print(document_word_embeddings.shape)
    # tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

    document_embeddings=np.zeros((len(tokenized_paded_documents),300))
    words=tfidfvectoriser.get_feature_names()
    #print(words)
    for i in range(len(document_word_embeddings)):
        for j in range(len(words)):  
            document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
            #print (document_embeddings[i])
    document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)
    #print(document_embeddings)

    # tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

    document_embeddings=np.zeros((len(tokenized_paded_documents),300))
    words=tfidfvectoriser.get_feature_names()

    for i in range(len(document_word_embeddings)):
        for j in range(len(words)):
            document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]

    document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)

            
    pairwise_similarities=cosine_similarity(document_embeddings)
    most_similar(0,pairwise_similarities,'Cosine Similarity')
    def most_similar(doc_id,similarity_matrix,matrix):
        print (f'Original: {documents_df.iloc[doc_id]["documents"]}')
        #print ('\n')
        #print (f'Similar Documents using {matrix}:')
        if matrix=='Cosine Similarity':
            similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
        elif matrix=='Euclidean Distance':
            similar_ix=np.argsort(similarity_matrix[doc_id])
        for ix in similar_ix:
            if ix==doc_id:
                continue
            print('\n')
            print (f'Document: {documents_df.iloc[ix]["documents"]}')
            print (f'{matrix} : {similarity_matrix[doc_id][ix]}')
            results2.append(similarity_matrix[0][1])
            results2.append(similarity_matrix[0][2])
            results2.append(similarity_matrix[0][3])
            
            #results2.append(similarity_matrix[doc_id][ix] )
    m=m+3


Original: Mark is at the park. His dog Max, is with him. Mark flies his kite and plays with max. They run and jump in the sun. Then Mark goes to the pond to see the ducks.


Document: Mark is playing with his dog, Max, at the park. He flies his kite, and he runs and jumps in the sun with max. Mark then sees the ducks in the pond.
Cosine Similarity : 0.9332349216051101


Document: Mark and his dog are playing under the sun at the park. They fly their kite, and then go to see the ducks in the pond. 
Cosine Similarity : 0.8399450147025951


Document: Mark is in the park. He is playing with his dog. They are also flying  kite. Mark then sees ducks in the river. 
Cosine Similarity : 0.7645549720561654
Original: Sir Isaac Newton was seated in his garden on a summer’s evening when
he saw an apple fall from a tree. He began to think, and, in trying to find
out why the apple fell, discovered how the earth, sun, moon, and stars
are kept in their places. 


Document: Sir Isaac Newton was in his g

Original: You can define and choose the voice profile that suits your organization and quickly adjust to changes in voice needs without needing to record new phrases.


Document: Define and choose the voice profile of your organization and quickly adjust to changes in voice needs without needing to record new phrases.
Cosine Similarity : 0.9870539824925915


Document: You can choose the voice profile that suits your organization and changes in voice needs without needing to record.
Cosine Similarity : 0.8940065426752154


Document: You can define and choose the voice profile that suits your organization.
Cosine Similarity : 0.7683030070678123
Original: Artificial intelligence is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by humans or animals


Document: Artificial intelligence is intelligence demonstrated by machines, contrary to the natural intelligence displayed by humans or animals.
Cosine Similarity : 0.9473739341543653


Document: Artif

Original: This whole process by which machines can be used to work for us has been called ‘automation’.


Document: Process which machines use to work for us has been called ‘automation’.
Cosine Similarity : 0.8922685840191454


Document: The process where machines can be used to work has been called ‘automation’.
Cosine Similarity : 0.8800904165140222


Document: The complete process by which machines can work for us has been called ‘automation’.
Cosine Similarity : 0.8579714358969717
Original: The earth is the only planet of our solar system so far known which contains plenty of water and this water has made our earth colourful, pulsating with life of a vast variety.


Document: The earth is the only planet in our solar system which contains plenty of water. This water has turned our earth colourful with life of a huge variety.
Cosine Similarity : 0.9222554426915126


Document: The earth is the only planet of our solar system so far known which contains plenty of water.
Cosine Simila

Original: a cube is a three-dimensional solid object bounded by six square faces, facets or sides, with three meeting at each vertex.


Document: a cube is a three-dimensional solid object
Cosine Similarity : 0.7219538906796936


Document: a symmetrical three-dimensional shape, either solid or hollow, contained by six equal squares
Cosine Similarity : 0.6961711265474009


Document: A cube number is the result when a number has been multiplied by itself twice. The symbol for cubed is 3
Cosine Similarity : 0.42081348889817255
Original: In chemistry and manufacturing, electrolysis is a technique that uses direct electric current to drive an otherwise non-spontaneous chemical reaction.


Document: In chemistry and manfacturing, electrolycis is a techniqe that uses direct electric curent to drive an otherwise non-spontanous chemical reaction.
Cosine Similarity : 0.8441534746793149


Document: chemical decomposition produced by passing an electric current through a liquid or solution contain

Original: any substance composed of identical molecules consisting of atoms of two or more chemical elements.


Document: a group of atoms bonded together, representing the smallest fundamental unit of a chemical compound that can take part in a chemical reaction
Cosine Similarity : 0.7119160427344577


Document: a thing that is composed of two or more separate elements
Cosine Similarity : 0.5717674147256901


Document: any thing made of identical molicules concisting of atams of two or more chemecal elements.
Cosine Similarity : 0.4945398498273337
Original: a group of atoms bonded together, representing the smallest fundamental unit of a chemical compound that can take part in a chemical reaction


Document: A molecule is an electrically neutral group of two or more atoms held together by chemical bonds.
Cosine Similarity : 0.7199576070935316


Document: a group of atams bounded together, representing the smalest fundamntal unite of a chemecal compound that can take part in a chemecal

Original: The submontane plateau. Lying south of the northern mountain rampart, the submontane plateau has four distinct divisions—the Trans-Indus plains, the Potwar Plateau, the Salt Range, and the Sialkot region.


Document: The submontane level. Lying south of the northern mountain defense, the submontane level has four particular divisions—the Trans-Indus fields, the Potwar Plateau, the Salt Range, and the Sialkot locale.
Cosine Similarity : 0.8541010661233802


Document: The submontae level. Lying lying  south of the northen mountain defense, the submontae level submontae level  has four particulr divisions—the Trns-Indus fields, the Potwar Plateau, the Salt Range, and the Sialkot local.
Cosine Similarity : 0.7927335670415697


Document: Pakistan is divided into three major geographic areas: the northern highlands; the Indus River plain, with two major subdivisions corresponding roughly to the provinces of Punjab and Sindh; and the Balochistan Plateau. Some geographers designate a

In [ ]:
print(results2)

[0.9332349216051101, 0.7645549720561654, 0.8399450147025951, 0.9332349216051101, 0.7645549720561654, 0.8399450147025951, 0.9332349216051101, 0.7645549720561654, 0.8399450147025951, 0.9021493274048592, 0.9058142075025011, 0.7845166755104833, 0.9021493274048592, 0.9058142075025011, 0.7845166755104833, 0.9021493274048592, 0.9058142075025011, 0.7845166755104833, 0.9141843439272872, 0.8908089986329026, 0.8562993094266347, 0.9141843439272872, 0.8908089986329026, 0.8562993094266347, 0.9141843439272872, 0.8908089986329026, 0.8562993094266347, 0.9425963316564767, 0.9004328700174342, 0.8507055266279, 0.9425963316564767, 0.9004328700174342, 0.8507055266279, 0.9425963316564767, 0.9004328700174342, 0.8507055266279, 0.87752505698724, 0.8782674933856831, 0.8775182614256849, 0.87752505698724, 0.8782674933856831, 0.8775182614256849, 0.87752505698724, 0.8782674933856831, 0.8775182614256849, 0.7738084746537435, 0.8497153433338057, 0.8040799088392497, 0.7738084746537435, 0.8497153433338057, 0.804079908839

In [ ]:
df=pd.DataFrame(results2)
df

,0
0,0.933235
1,0.764555
2,0.839945
3,0.933235
4,0.764555
...,...
895,0.714965
896,0.428098
897,0.794784
898,0.714965


In [ ]:
df.to_excel("C:/Users/HP/Desktop/lowest-common-subsumer/word2vec3.xlsx") 

In [ ]:
df.info

<bound method DataFrame.info of             0
0    0.933235
1    0.764555
2    0.839945
3    0.933235
4    0.764555
..        ...
895  0.714965
896  0.428098
897  0.794784
898  0.714965
899  0.428098

[900 rows x 1 columns]>

In [ ]:
i=0
final=[]
while i<(len(df.index))-8:
    for j in range(3):
        final.append(results2[i+j])
        print(results2[i+j])
    i=i+9

0.9332349216051101
0.7645549720561654
0.8399450147025951
0.9021493274048592
0.9058142075025011
0.7845166755104833
0.9141843439272872
0.8908089986329026
0.8562993094266347
0.9425963316564767
0.9004328700174342
0.8507055266279
0.87752505698724
0.8782674933856831
0.8775182614256849
0.7738084746537435
0.8497153433338057
0.8040799088392497
0.8325710263251893
0.7951477988423291
0.7515081436307411
0.9168121006465435
0.9319826160796062
0.6227032548007857
0.9030451291511954
0.8545939936640573
0.7059315982051011
0.869994274357768
0.8608749461638108
0.7958247069729321
0.9389631985161024
0.8204371207413216
0.9762029685360412
0.8166087626593049
0.9094292509341537
0.8876304180569458
0.9596140818340594
0.8422118253049861
0.7898866706838947
0.9156710690389644
0.799716023461408
0.8347826528144943
0.9906185890493748
0.9765967575201846
0.9549664823001345
0.8822058992683881
1.0000000000000004
0.9554551705888208
0.9196449557797166
0.8466929489962106
0.9929503716271647
0.9883157737622776
0.8500228164931594


In [ ]:
word2vec=pd.DataFrame(final)

In [ ]:
word2vec.to_excel("C:/Users/HP/Desktop/lowest-common-subsumer/word.xlsx") 